In [1]:
import datetime, json, os, time
import pandas as pd

from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials

from ratelimit import limits, sleep_and_retry

# Load and set credentials
with open('credentials.json') as json_file:
    credentials = json.load(json_file)

LUIS_RUNTIME_KEY=credentials['LUIS_RUNTIME_KEY']
LUIS_RUNTIME_ENDPOINT=credentials['LUIS_RUNTIME_ENDPOINT']
LUIS_RUNTIME_APPID=credentials['LUIS_RUNTIME_APPID']

# Load data into dataframe
df = pd.read_csv("example.csv",  index_col='id')

# view df from file
df.head()

,text
id,
1,the fingerprint reader on my phone no longer w...
2,I am unable to login to the application
3,this is an example where someone has many issu...
4,i cant login and i'm really annoyed about it. ...
5,everything worked brilliantly and i can't wait...


In [2]:
# Instantiate a LUIS runtime client
clientRuntime = LUISRuntimeClient(LUIS_RUNTIME_ENDPOINT, CognitiveServicesCredentials(LUIS_RUNTIME_KEY))

In [3]:
# Define functions for prediction and entity parsing from response

@sleep_and_retry
@limits(calls=5, period=1)
def predict(app_id, slot_name, query):
    request = { "query" : str(query) }
    # Note be sure to specify, using the slot_name parameter, whether your application is in staging or production.
    resp = clientRuntime.prediction.get_slot_prediction(app_id=app_id, slot_name=slot_name, prediction_request=request)
    return (resp)

def get_all_values(d):
    if isinstance(d, dict):
        for v in d.values():
            yield from get_all_values(v)
    elif isinstance(d, list):
        for v in d:
            yield from get_all_values(v)
    else:
        yield d 

def get_results(resp):
    return ','.join(get_all_values(list(resp.prediction.entities.keys()))) , resp.prediction.sentiment


In [4]:
# Create new column with entities from LUIS
df[['entities', 'sentiment']] = df.apply(lambda row: pd.Series(get_results(predict(LUIS_RUNTIME_APPID, 'production', row['text']))), axis=1)

In [5]:
# df with entities column
df.head()

,text,entities,sentiment
id,,,
1,the fingerprint reader on my phone no longer w...,Fingerprint,None
2,I am unable to login to the application,Login,None
3,this is an example where someone has many issu...,"Fingerprint,Login",None
4,i cant login and i'm really annoyed about it. ...,Login,None
5,everything worked brilliantly and i can't wait...,,None
